In [1]:
import pandas as pd
import numpy as np
import datetime
basetime = datetime.datetime(year=1970, month=1, day=1, hour=1)

%run get_odds.py
%run Helper.py

## Update 2020/2021 data

In [2]:
# football data
url = "https://www.football-data.co.uk/mmz4281/"
ext = ['E0', 'E1', 'E2', 'E3', 'SP1', 'SP2', 'SC0', 'D1', 'D2', 
       'I1', 'SP1', 'SP2', 'F1', 'N1', 'B1', 'P1', 'T1', 'G1']
year = 2021
year = str(year)
for league in ext:
    full_url = url + year + '/' + league + '.csv'
    #display(full_url)
    r = requests.get(full_url, allow_redirects=True)
    open('AutoData/'+year+league+'.csv', 'wb').write(r.content)
    data = read_football_data('AutoData/'+year+league+'.csv')
    data.to_csv('AutoData/'+year+league+'.csv', index=False)

# betting data 
ext = ['E0', 'SP1', 'SC0', 'D1', 'I1', 'F1', 'N1', 'B1', 'P1']
year = 2021
prev_year = str(year-1)
year = str(year)
int(year[2:])
year_string = prev_year[2:]+year[2:]

for league in ext:
    full_url = url + year_string + '/' + league + '.csv'
    #display(full_url)
    r = requests.get(full_url, allow_redirects=True)
    open('BettingData/'+year+league+'.csv', 'wb').write(r.content)
    data = read_betting_data('BettingData/'+year+league+'.csv')
    data.to_csv('BettingData/'+year+league+'.csv', index=False)

### Create dataframe containing new recommended bets

In [3]:
new_bets = pd.DataFrame(columns = ['SEA', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'Type', 'Odds', 'Bookie', 'Stake'])

## Premier League

In [4]:
# instantiate and train up to 2019/2020 season
PL = league_fast()
PL.train_all(league_str='E0', league_below='E1')
# 2020/2021 season
new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/E0.csv")
PL.new_season(teams_out=['Bournemouth', 'Norwich', 'Watford'], teams_promoted_in=['Fulham','Leeds', 'West Brom'])
PL.train(new_data)

Training Complete


In [5]:
PL_odds = get_odds()

Remaining requests 477
Used requests 23


In [6]:
convert_names = {'Arsenal': 'Arsenal', 'Aston Villa': 'Aston Villa',
                 'Brighton and Hove Albion': 'Brighton', 'Burnley': 'Burnley',
                 'Chelsea': 'Chelsea', 'Crystal Palace': 'Crystal Palace',
                 'Everton': 'Everton', 'Fulham': 'Fulham', 'Leeds United': 'Leeds',
                 'Leicester City': 'Leicester', 'Liverpool': 'Liverpool',         
                 'Manchester City': 'Man City', 'Manchester United': 'Man United',
                 'Newcastle United': 'Newcastle', 'Sheffield United': 'Sheffield United',
                 'Southampton': 'Southampton', 'Tottenham Hotspur': 'Tottenham',
                 'West Bromwich Albion': 'West Brom', 'West Ham United': 'West Ham',
                 'Wolverhampton Wanderers': 'Wolves'}
# decide whether we should bet
thresh = 0.04
for match in PL_odds[:10]:
    HomeTeam = convert_names[match['match'][0]]
    AwayTeam = convert_names[match['match'][1]]
    model_probs = PL.predict(HomeTeam, AwayTeam)['outcomes']
    betting_probs = match['Probs']
    # identify betting opportunities
    for j in [0]:
        if model_probs[j] - betting_probs[j] > thresh:
            print(HomeTeam + ' vs ' + AwayTeam)
            outcomes = [HomeTeam, 'Draw', AwayTeam]
            new_bets = new_bets.append(pd.DataFrame(data={'SEA':2021, 'Div':'E0', 
                                        'Date':match['time'], 
                                       'HomeTeam':HomeTeam, 'AwayTeam':AwayTeam, 'Type':'H', 
                                       'Odds':match['Odds'][j], 'Bookie':match['Bookie'][j], 'Stake':10}, index=[0]), ignore_index=True)
            print('Bet on ' + outcomes[j] + ' with ' + match['Bookie'][j]+'. Odds of ' + str(match['Odds'][j])) 
            print('Difference in probs: ' + str(model_probs[j] - betting_probs[j]))
            print('Betting Probs: ' + str(betting_probs))
            print('Model Probs: ' + str(model_probs))
            print()

West Brom vs Fulham
Bet on West Brom with Unibet. Odds of 3.45
Difference in probs: 0.08972899143135521
Betting Probs: [0.2898550724637681, 0.30303030303030304, 0.4098360655737705]
Model Probs: [0.37958406389512334, 0.2971239389354938, 0.3232919971693828]

Crystal Palace vs Wolves
Bet on Crystal Palace with Betfair. Odds of 3.1
Difference in probs: 0.06958765831089625
Betting Probs: [0.3225806451612903, 0.3125, 0.3676470588235294]
Model Probs: [0.39216830347218656, 0.2922965775559599, 0.3155351189718535]

Arsenal vs Man United
Bet on Arsenal with Betfair. Odds of 2.98
Difference in probs: 0.0946853036245009
Betting Probs: [0.33557046979865773, 0.28169014084507044, 0.3846153846153846]
Model Probs: [0.4302557734231586, 0.25444557089077585, 0.31529865568606563]

Southampton vs Aston Villa
Bet on Southampton with Betfair. Odds of 3.0
Difference in probs: 0.19143658364678123
Betting Probs: [0.3333333333333333, 0.27027027027027023, 0.40322580645161293]
Model Probs: [0.5247699169801145, 0.243

## Spanish League

In [7]:
# initialise league
SP1 = league_fast()
# train on all data up to 2019/2020 season
SP1.train_all(league_str='SP1', league_below='SP2', SEA=range(1998, 2021))
# train on 2020/2021 season
SP1.new_season(teams_out=['Espanol', 'Mallorca', 'Leganes'], teams_promoted_in=['Cadiz', 'Huesca', 'Elche'])
SP1.train(pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/SP1.csv"))

Training Complete


In [8]:
SP_odds = get_odds('soccer_spain_la_liga')

Remaining requests 476
Used requests 24


In [9]:
convert_names = {'Alavés':'Alaves', 'Athletic Bilbao':'Ath Bilbao', 'Atlético Madrid':'Ath Madrid', 
                 'Barcelona':'Barcelona', 'Real Betis':'Betis', 'Cádiz CF':'Cadiz', 'Celta Vigo':'Celta', 
                 'Eibar':'Eibar', 'Elche CF':'Elche', 'Getafe':'Getafe', 'Granada CF':'Granada', 'Huesca':'Huesca',
                 'Levante':'Levante', 'CA Osasuna':'Osasuna', 'Real Madrid':'Real Madrid', 'Sevilla':'Sevilla', 
                 'Real Sociedad':'Sociedad', 'Valencia':'Valencia', 'Valladolid':'Valladolid',  'Villarreal':'Villarreal'}

# decide whether we should bet
thresh = 0.05
for match in SP_odds[:10]:
    HomeTeam = convert_names[match['match'][0]]
    AwayTeam = convert_names[match['match'][1]]
    model_probs = SP1.predict(HomeTeam, AwayTeam)['outcomes']
    betting_probs = match['Probs']
    symbol = ['H', 'D', 'A']
    # identify betting opportunities
    for j in [0]:
        if model_probs[j] - betting_probs[j] > thresh:
            print(HomeTeam + ' vs ' + AwayTeam)
            outcomes = [HomeTeam, 'Draw', AwayTeam]
            new_bets = new_bets.append(pd.DataFrame(data={'SEA':2021, 'Div':'SP1', 
                                        'Date':match['time'], 
                                       'HomeTeam':HomeTeam, 'AwayTeam':AwayTeam, 'Type':symbol[j], 
                                       'Odds':match['Odds'][j], 'Bookie':match['Bookie'][j], 'Stake':10}, index=[0]), ignore_index=True)
            
            print('Bet on ' + outcomes[j] + ' with ' + match['Bookie'][j]+'. Odds of ' + str(match['Odds'][j])) 
            print('Difference in probs: ' + str(model_probs[j] - betting_probs[j]))
            print('Betting Probs: ' + str(betting_probs))
            print('Model Probs: ' + str(model_probs))
            print()

Valladolid vs Huesca
Bet on Valladolid with Betfair. Odds of 2.48
Difference in probs: 0.08139285347984976
Betting Probs: [0.40322580645161293, 0.31746031746031744, 0.2857142857142857]
Model Probs: [0.4846186599314627, 0.3004205055790838, 0.2149608344894537]

Eibar vs Sevilla
Bet on Eibar with Betfair. Odds of 4.2
Difference in probs: 0.06861081642706163
Betting Probs: [0.23809523809523808, 0.29411764705882354, 0.4807692307692307]
Model Probs: [0.3067060545222997, 0.2962830675460102, 0.3970108779316901]

Villarreal vs Sociedad
Bet on Villarreal with Betfair. Odds of 2.5
Difference in probs: 0.09758075247387255
Betting Probs: [0.4, 0.3076923076923077, 0.29850746268656714]
Model Probs: [0.49758075247387257, 0.24885079171758304, 0.25356845580854437]

Granada vs Celta
Bet on Granada with Betfair. Odds of 2.88
Difference in probs: 0.14603401648736136
Betting Probs: [0.3472222222222222, 0.30303030303030304, 0.3597122302158274]
Model Probs: [0.49325623870958357, 0.2486436260642107, 0.25810013

## German League

In [10]:
# initialise league
D1 = league_fast()
# train on all data up to 2019/2020 season
D1.train_all(league_str='D1', SEA=range(1996, 2021))
# train on 2020/2021 season
D1.new_season(teams_out=['Fortuna Dusseldorf', 'Paderborn'], teams_promoted_in=['Bielefeld', 'Stuttgart'])
D1.train(pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/D1.csv"))

Training Complete


In [11]:
D1_odds = get_odds('soccer_germany_bundesliga')

Remaining requests 475
Used requests 25


In [12]:
convert_names = {'Arminia Bielefeld':'Bielefeld', 'Augsburg':'Augsburg', 'Bayer Leverkusen':'Leverkusen', 
                 'Bayern Munich':'Bayern Munich', 'Borussia Dortmund':'Dortmund', 
                 'Borussia Monchengladbach':"M'gladbach", 'Eintracht Frankfurt':'Ein Frankfurt', 'FC Koln':'FC Koln', 
                 'FSV Mainz 05':'Mainz', 'Hertha Berlin':'Hertha', 'RB Leipzig':'RB Leipzig', 'Schalke 04':'Schalke 04', 
                 'SC Freiburg':'Freiburg', 'TSG Hoffenheim':'Hoffenheim', 'Union Berlin':'Union Berlin', 
                 'VfB Stuttgart':'Stuttgart', 'VfL Wolfsburg':'Wolfsburg', 'Werder Bremen':'Werder Bremen'}

# decide whether we should bet
thresholds = [0.08, 1, 0.04]
for match in D1_odds[:10]:
    HomeTeam = convert_names[match['match'][0]]
    AwayTeam = convert_names[match['match'][1]]
    model_probs = D1.predict(HomeTeam, AwayTeam)['outcomes']
    betting_probs = match['Probs']
    symbol = ['H', 'D', 'A']
    # identify betting opportunities
    for j in range(3):
        if model_probs[j] - betting_probs[j] > thresholds[j]:
            print(HomeTeam + ' vs ' + AwayTeam)
            outcomes = [HomeTeam, 'Draw', AwayTeam]
            new_bets = new_bets.append(pd.DataFrame(data={'SEA':2021, 'Div':'D1', 
                                        'Date':match['time'], 
                                       'HomeTeam':HomeTeam, 'AwayTeam':AwayTeam, 'Type':symbol[j], 
                                       'Odds':match['Odds'][j], 'Bookie':match['Bookie'][j], 'Stake':10}, index=[0]), ignore_index=True)
            
            print('Bet on ' + outcomes[j] + ' with ' + match['Bookie'][j]+'. Odds of ' + str(match['Odds'][j])) 
            print('Difference in probs: ' + str(model_probs[j] - betting_probs[j]))
            print('Betting Probs: ' + str(betting_probs))
            print('Model Probs: ' + str(model_probs))
            print()

Ein Frankfurt vs Hertha
Bet on Ein Frankfurt with Betfair. Odds of 1.85
Difference in probs: 0.08946164261234724
Betting Probs: [0.5405405405405405, 0.2531645569620253, 0.2127659574468085]
Model Probs: [0.6300021831528877, 0.19291323955558132, 0.17708457729153088]

Union Berlin vs M'gladbach
Bet on Union Berlin with Betfair. Odds of 3.45
Difference in probs: 0.08201340532222429
Betting Probs: [0.2898550724637681, 0.28169014084507044, 0.4347826086956522]
Model Probs: [0.3718684777859924, 0.2508986480825062, 0.3772328741315013]

Werder Bremen vs Schalke 04
Bet on Werder Bremen with Betfair. Odds of 2.0
Difference in probs: 0.10743466227052734
Betting Probs: [0.5, 0.27027027027027023, 0.23809523809523808]
Model Probs: [0.6074346622705273, 0.21930975151314216, 0.1732555862163306]



## French League
We do not bet in the French League as our exploratory analysis found this to be unprofitable

In [13]:
# instantiate and train up to 2019/2020 season
F1 = league_fast()
F1.train_all(league_str='F1', league_below='F2', SEA=range(2003, 2021))
# 2020/2021 season
new_data = pd.read_csv("https://www.football-data.co.uk/mmz4281/2021/F1.csv")
F1.new_season(teams_out=['Amiens', 'Toulouse'], teams_promoted_in=['Lorient', 'Lens'])
F1.train(new_data)

Training Complete


In [14]:
F1_odds = get_odds('soccer_france_ligue_one')

Remaining requests 474
Used requests 26


In [15]:
convert_names = {'Angers':'Angers', 'Bordeaux':'Bordeaux', 'Brest':'Brest', 'Dijon': 'Dijon',  
                 'RC Lens':'Lens', 'Lille':'Lille', 'Lorient':'Lorient', 'Lyon':'Lyon', 'Marseille':'Marseille',
                 'Metz':'Metz', 'AS Monaco':'Monaco', 'Montpellier':'Montpellier', 'Nantes':'Nantes', 
                 'Nice':'Nice', 'Nîmes Olympique':'Nimes', 'Paris Saint Germain':'Paris SG', 
                 'Stade de Reims':'Reims', 'Rennes':'Rennes', 'Saint Etienne':'St Etienne', 
                 'Strasbourg':'Strasbourg', 'Toulouse':'Toulouse'}

# decide whether we should bet
thresh = 1
for match in F1_odds[:10]:
    HomeTeam = convert_names[match['match'][0]]
    AwayTeam = convert_names[match['match'][1]]
    model_probs = F1.predict(HomeTeam, AwayTeam)['outcomes']
    betting_probs = match['Probs']
    # identify betting opportunities
    for j in range(3):
        if model_probs[j] - betting_probs[j] > thresh:
            print(HomeTeam + ' vs ' + AwayTeam)
            outcomes = [HomeTeam, 'Draw', AwayTeam]
            print('Bet on ' + outcomes[j] + ' with ' + match['Bookie'][j]+'. Odds of ' + str(match['Odds'][j])) 
            print('Difference in probs: ' + str(model_probs[j] - betting_probs[j]))
            print('Betting Probs: ' + str(betting_probs))
            print('Model Probs: ' + str(model_probs))
            print()

## Scottish League

In [16]:
SC0 = league_fast()
SC0.train_all(league_str='SC0', SEA=range(2001, 2022))

Training Complete


In [17]:
SC0_odds = get_odds('soccer_spl')

Remaining requests 473
Used requests 27


In [18]:
convert_names = {'Aberdeen':'Aberdeen', 'Celtic':'Celtic', 'Dundee United':'Dundee United',
                 'Hamilton Academical':'Hamilton', 'Hibernian':'Hibernian', 'Kilmarnock':'Kilmarnock', 
                 'Livingston':'Livingston', 'Motherwell':'Motherwell',  'Rangers':'Rangers', 
                 'Ross County':'Ross County', 'St Johnstone':'St Johnstone', 'St Mirren':'St Mirren'}

# decide whether we should bet
thresholds = [1, 0.05, 0.05]
for match in SC0_odds[:10]:
    HomeTeam = convert_names[match['match'][0]]
    AwayTeam = convert_names[match['match'][1]]
    model_probs = SC0.predict(HomeTeam, AwayTeam)['outcomes']
    betting_probs = match['Probs']
    symbol = ['H', 'D', 'A']
#     print(HomeTeam, AwayTeam)
#     print('Betting Probs: ' + str(betting_probs))
#     print('Model Probs: ' + str(model_probs))
#     print()
    # identify betting opportunities
    for j in range(3):
        if model_probs[j] - betting_probs[j] > thresh:
            print(HomeTeam + ' vs ' + AwayTeam)
            outcomes = [HomeTeam, 'Draw', AwayTeam]
            new_bets = new_bets.append(pd.DataFrame(data={'SEA':2021, 'Div':'SC0', 
                                        'Date':match['time'], 
                                       'HomeTeam':HomeTeam, 'AwayTeam':AwayTeam, 'Type':symbol[j], 
                                       'Odds':match['Odds'][j], 'Bookie':match['Bookie'][j], 'Stake':10}, index=[0]), 
                                       ignore_index=True)
            print('Bet on ' + outcomes[j] + ' with ' + match['Bookie'][j]+'. Odds of ' + str(match['Odds'][j])) 
            print('Difference in probs: ' + str(model_probs[j] - betting_probs[j]))
            print('Betting Probs: ' + str(betting_probs))
            print('Model Probs: ' + str(model_probs))
            print()

### Put new bets into tracking dataframe and save 

In [19]:
new_bets

,SEA,Div,Date,HomeTeam,AwayTeam,Type,Odds,Bookie,Stake
0,2021,E0,2021-01-30 16:00:00,West Brom,Fulham,H,3.45,Unibet,10
1,2021,E0,2021-01-30 16:00:00,Crystal Palace,Wolves,H,3.10,Betfair,10
2,2021,E0,2021-01-30 18:30:00,Arsenal,Man United,H,2.98,Betfair,10
3,2021,E0,2021-01-30 21:00:00,Southampton,Aston Villa,H,3.00,Betfair,10
4,2021,E0,2021-01-31 15:00:00,Leicester,Leeds,H,1.87,Betfair,10
5,2021,SP1,2021-01-29 21:00:00,Valladolid,Huesca,H,2.48,Betfair,10
6,2021,SP1,2021-01-30 14:00:00,Eibar,Sevilla,H,4.20,Betfair,10
7,2021,SP1,2021-01-30 21:00:00,Villarreal,Sociedad,H,2.50,Betfair,10
8,2021,SP1,2021-01-31 18:30:00,Granada,Celta,H,2.88,Betfair,10
9,2021,SP1,2021-01-31 21:00:00,Barcelona,Ath Bilbao,H,1.45,Betclic,10


In [20]:
new_bets['Stake'] = 0
new_bets['FTR'] = 0
betting_df = pd.read_csv('Betting Strategy Tracker.csv', index_col=False)
betting_df['Date'] = pd.to_datetime(betting_df['Date'], dayfirst=True)
betting_df = betting_df.append(new_bets, ignore_index=True)
betting_df = betting_df[betting_df.duplicated(subset=['SEA', 'HomeTeam', 'AwayTeam'])==False]

In [21]:
betting_df = betting_df.sort_values('Date', ignore_index=True)

In [22]:
betting_df

,SEA,Div,Date,HomeTeam,AwayTeam,Type,Odds,Bookie,Stake,FTR,Returns
0,2021,SP1,2020-10-17 12:00:00,Granada,Sevilla,H,4.30,Betfair,10,H,43.0
1,2021,D1,2020-10-17 14:30:00,Hoffenheim,Dortmund,H,4.40,Betfair,10,A,0.0
2,2021,D1,2020-10-17 14:30:00,Mainz,Leverkusen,H,5.50,William Hill,10,A,0.0
3,2021,D1,2020-10-17 14:30:00,Hertha,Stuttgart,H,2.12,Betfair,10,A,0.0
4,2021,D1,2020-10-17 14:30:00,Augsburg,RB Leipzig,H,6.00,Unibet,10,A,0.0
...,...,...,...,...,...,...,...,...,...,...,...
191,2021,SP1,2021-01-30 21:00:00,Villarreal,Sociedad,H,2.50,Betfair,0,0,NaN
192,2021,E0,2021-01-30 21:00:00,Southampton,Aston Villa,H,3.00,Betfair,0,0,NaN
193,2021,E0,2021-01-31 15:00:00,Leicester,Leeds,H,1.87,Betfair,0,0,NaN
194,2021,SP1,2021-01-31 18:30:00,Granada,Celta,H,2.88,Betfair,0,0,NaN


In [23]:
for i in range(betting_df.shape[0]):
    bet = betting_df.iloc[i]
    date = bet['Date']
    if pd.to_datetime(date, dayfirst=True) > datetime.datetime.now():
        break
    div = bet['Div']
    sea = bet['SEA']
    # lookup data
    filename = 'AutoData/' + str(sea) + str(div) + '.csv'
    data = pd.read_csv(filename)
    h_index = data['HomeTeam']==bet['HomeTeam']
    a_index = data['AwayTeam']==bet['AwayTeam']
    index = h_index & a_index
    if sum(index)==0:
        continue
    index = np.where(index)[0][0]
    betting_df.loc[i, 'Stake'] = 10
    betting_df.loc[i, 'FTR'] = data.loc[index, 'FTR']
    
betting_df['Returns'] = betting_df['Stake'] * (betting_df['Type']==betting_df['FTR']) * betting_df['Odds']

In [24]:
betting_df.to_csv('Betting Strategy Tracker.csv', index=False)

In [25]:
pd.read_csv('Betting Strategy Tracker.csv')

,SEA,Div,Date,HomeTeam,AwayTeam,Type,Odds,Bookie,Stake,FTR,Returns
0,2021,SP1,2020-10-17 12:00:00,Granada,Sevilla,H,4.30,Betfair,10,H,43.0
1,2021,D1,2020-10-17 14:30:00,Hoffenheim,Dortmund,H,4.40,Betfair,10,A,0.0
2,2021,D1,2020-10-17 14:30:00,Mainz,Leverkusen,H,5.50,William Hill,10,A,0.0
3,2021,D1,2020-10-17 14:30:00,Hertha,Stuttgart,H,2.12,Betfair,10,A,0.0
4,2021,D1,2020-10-17 14:30:00,Augsburg,RB Leipzig,H,6.00,Unibet,10,A,0.0
...,...,...,...,...,...,...,...,...,...,...,...
191,2021,SP1,2021-01-30 21:00:00,Villarreal,Sociedad,H,2.50,Betfair,0,0,0.0
192,2021,E0,2021-01-30 21:00:00,Southampton,Aston Villa,H,3.00,Betfair,0,0,0.0
193,2021,E0,2021-01-31 15:00:00,Leicester,Leeds,H,1.87,Betfair,0,0,0.0
194,2021,SP1,2021-01-31 18:30:00,Granada,Celta,H,2.88,Betfair,0,0,0.0


In [26]:
staked = sum(betting_df['Stake'])
returned = sum(betting_df['Returns'])
print('Staked: ' + str(staked))
print('Returned: ' + str(returned))

by_league = pd.DataFrame(columns=['Div', 'Staked', 'Returned'])
for league in np.unique(betting_df['Div']):
    league_data = betting_df[betting_df['Div']==league]
    by_league = by_league.append(pd.DataFrame({'Div':league, 'Staked':sum(league_data['Stake']), 'Returned':sum(league_data['Returns'])}, 
                                 index=[0]), ignore_index=True)
by_league['Percentage Profit'] =  np.array(((by_league['Returned']/by_league['Staked'])-1)*100)
display(by_league)

Staked: 1810
Returned: 1533.9000000000003


,Div,Staked,Returned,Percentage Profit
0,D1,410,226.0,-44.878
1,E0,840,744.3,-11.3929
2,SP1,560,563.6,0.642857


In [27]:
month = betting_df[betting_df['Date']>'2020-12-1']

In [28]:
staked = sum(month['Stake'])
returned = sum(month['Returns'])
print('Staked: ' + str(staked))
print('Returned: ' + str(returned))

by_league = pd.DataFrame(columns=['Div', 'Staked', 'Returned'])
for league in np.unique(month['Div']):
    league_data = month[month['Div']==league]
    by_league = by_league.append(pd.DataFrame({'Div':league, 'Staked':sum(league_data['Stake']), 'Returned':sum(league_data['Returns'])}, 
                                 index=[0]), ignore_index=True)
by_league['Percentage Profit'] =  np.array(((by_league['Returned']/by_league['Staked'])-1)*100)
display(by_league)

Staked: 990
Returned: 841.3999999999999


,Div,Staked,Returned,Percentage Profit
0,D1,230,187.7,-18.3913
1,E0,480,415.3,-13.4792
2,SP1,280,238.4,-14.8571
